# Simple Agent Example
- tested on jupyter lab on wsl in windows 11. probably works okay most places.
- will require a running ollama server that has qwen3:1.7b installed (or whatever agent you decide)
- you don't really even need the langchain_core.messages stuff, as long as there's some kind of a message history in a list it can access, it should work fine.
- Relevant docs:
    * https://langchain-ai.github.io/langgraph/reference/agents
    * https://ollama.com/library/qwen3:1.7b

In [ ]:
%pip install langgraph
%pip install -qU langchain-ollama langchain-community duckduckgo-search

import asyncio, random
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import MessagesState
from langchain_community.tools import DuckDuckGoSearchRun

from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
    trim_messages,
)
from langchain_core.messages.utils import count_tokens_approximately

In [ ]:
def tool_example():
    """
    tool_example is a demo agentic tool
    args: 
        - none
    returns:
        - a string of text. 
    """
    print("executing tool.")
    return "Demo tool run succesfully."

search = DuckDuckGoSearchRun()
llm = ChatOllama(model="qwen3:1.7b")
agent_prompt = """You are a helpful intelligent agent."""

agent = create_react_agent(
    model=llm,
    tools=[search, tool_example],
    prompt=agent_prompt
)


In [ ]:

## this holds the entire conversation history fed to the agent for now.
messages = []

username = input("User name: ")
greeting = f"Hello {username}. How can I help you today?"
print(greeting)

while True:
    try:
        user_input = input("User: ")
        messages = HumanMessage("<sender>"+username+"</sender><message>"+user_input+"</message>")
        result = agent.invoke({"messages": messages})
        for msg in result["messages"]:
            msg.pretty_print()
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        
    except EOFError:
        # fallback if input() is not available
        user_input = "Thank you for your service!"
        print("User: " + user_input)
        
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        break

